# Homework 1 for CS 329P

**Authors**: YOUR_NAMES

**Emails**: YOUR_IDS@stanford.edu

**Submission.** Please insert your names and emails above, save your code in this notebook, and explain what you are doing along with your findings in text cells. You can think of it as a technical report with code. Before submission, please use `Kernel -> Restart & Run All` in the Jupyter menu to verify your code is runnable and save all outputs. Afterwards, you can either upload your raw notebook (`hw1.ipynb`) or an exported PDF version to the `Homework 1` assignment in Canvas. 


In this homework, we will train a house sales price predictor on the data we scraped previously. The purpose of this homework is to let you practice different techniques that you can use to preprocess raw data. Your job is to obtain the best root mean squared logarithmic error (RMSLE) on the test dataset. To make your job easy, we provide sample code to train a model to report RMSLE and a list of ideas you can explore.

**Note**: You can use either local runtimes to complete this assignment, or a hosted runtime (with GPU) on Colab. The second option generally runs faster. If using a local runtime, make sure that your Python version is less than 3.9 but at least 3.6, or you may have issues installing Autogluon. If using a runtime hosted on Colab, you can use the File Explorer pane on the left to upload the `house_sales.ftr` file. Make sure to wait until the file finishes uploading before running the next code block.

Additionally, if using a local runtime, please refer to the [AG document](https://auto.gluon.ai/stable/index.html#installation) for info on how to install autogluon. 

## Prepare Data 

Let's first read in the dataset we used in our [Exploratory Data Analysis (EDA)](https://c.d2l.ai/stanford-cs329p/_static/notebooks/cs329p_notebook_eda.slides.html). Note that we use the [`feather` format](https://arrow.apache.org/docs/python/feather.html), which is faster to read than CSV but uses more disk space. The file `home_sales.ftr` can be downloaded from the Assignments folder in Canvas.

Just for your information, it is generated with:

```python
data = pd.read_csv('house_sales.zip', dtype='unicode')
data.to_feather('house_sales.ftr')
```

The following code needs at least 2GB memory. If using a local runtime, please make sure your machine has enough memory. 

In [ ]:

import pandas as pd
import numpy as np

data = pd.read_feather('house_sales.ftr')


In [ ]:
import scipy
import numpy as np
scipy.__version__, np.__version__

We select a few common columns to make our training fast. You need to select more columns to make your model more accurate. 

In [ ]:
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms']].copy()
# uncomment the below line to save memory
# del data

We copy the code from EDA to convert `Sold Price` to numerical values, which is our prediction target. We also remove examples whose prices are too high or too low.

In [ ]:
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )]

We use the house sales between 2021-2-15 and 2021-3-1 as our test data. You can use any example before 2021-2-15, but not after. In other words, we pretend we are launching our model on 2021-2-15 and testing it for 2 weeks. Here we only use sales in 2021 for fast training, but you can use more to improve accuracy. 

In [ ]:
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1)
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

Define our evaluation metric. 

In [ ]:
def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

## AutoGluon Baseline

We provide a baseline model trained by AutoGluon (AG). AG is an automl tool that performs automatic feature engineering, model selections, and ensemble. You are welcome to use any model and tool in achieving the best results possible in your homework. However, we recommend that you reuse the following training code so that you can focus on data preprocessing.

In [ ]:
from autogluon.tabular import TabularPredictor
    
label = 'Sold Price'    
predictor = TabularPredictor(label=label).fit(train)

Test the performance of each model. 

In [ ]:
predictor.leaderboard(test, silent=True)

Next, we compute the importance of each feature, along with several other metrics. It loooks like the `Sold On` feature is not very useful, likely because the houses in the test data were all sold late. You can choose to either remove such a feature, or find a way to extract a more useful presentation from it.

In [ ]:
predictor.feature_importance(test)

Finally, let's predict and evaluate the RMSLE.

In [ ]:
preds = predictor.predict(test.drop(columns=[label]))
rmsle(preds, test[label])

## Your Solution

Please include your solution in the following section. (You are welcome to edit and delete code in previous sections).

Your goal is to train a model using the features in the original dataset that minimizes the RMSLE on the validation dataset. While the naïve model achieves an RMSLE of ~0.3, it is possible to achieve an RMSLE of less than 0.08 on the same dataset.

Here is a list of ideas you could explore:

- More features: We only selected a small set of columns to use in training. You can add more, especially the ones we examined in EDA.
- Data type conversion: Most data columns are strings; you may need to convert them into numerical values.
- Data cleaning: There are NAN and outliers sprinkled throughout the dataset. You should find ways to selectively filter and remove them.
- More examples: We only included sales made in 2021; there is a large number of examples in previous years that you can also include.

Check data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_feather('house_sales.ftr')
data['Type'].value_counts()[0:20]

Data clean

In [ ]:
df=data.copy()
#按年份筛选
df['Type']=df['Type'].replace('Single Family','SingleFamily')
top_5_types = df['Type'].value_counts().index[:5]
filtered_df = df[df['Type'].isin(top_5_types)]
filtered_df=filtered_df[filtered_df['Year built']!='No Data']
filtered_df['Year built'] = pd.to_numeric(filtered_df['Year built'], errors='coerce')
filtered_df=filtered_df[filtered_df['Year built']>1800]
#加热和制冷方式缺失太多直接drop掉
filtered_df=filtered_df.drop('Heating',axis=1)
filtered_df=filtered_df.drop('Cooling',axis=1)
# 定义正则表达式模式，匹配Parking中包含 'Garage' 的类别
pattern = r'.*Garage.*'
# 将匹配的类别替换为 'Garage'
filtered_df['Parking'] = filtered_df['Parking'].replace(to_replace=pattern, value='Garage', regex=True)
filtered_df=filtered_df[filtered_df['Parking']!='No Data']
filtered_df['Parking']=filtered_df['Parking'].replace('0 spaces','None')
top_5_parking = filtered_df['Parking'].value_counts().index[:5]
filtered_df = filtered_df[filtered_df['Parking'].isin(top_5_parking)]
filtered_df['Parking']=filtered_df['Parking'].replace('None','No parking')
pattern = r'.*Ground Floor Bedroom.*'
filtered_df['Bedrooms'] = filtered_df['Bedrooms'].replace(to_replace=pattern, value='Ground Floor Bedroom', regex=True)
filtered_df['Bedrooms']=filtered_df['Bedrooms'].replace('Ground Floor Bedroom','1')
filtered_df = filtered_df.dropna(subset=['Bedrooms'])
filtered_df=filtered_df[filtered_df['Bedrooms'].str.isdigit()]
filtered_df['Bedrooms'] = pd.to_numeric(filtered_df['Bedrooms'], errors='coerce')
final_df = filtered_df[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms']].copy()
filtered_df['Bathrooms'].value_counts()

Data transformation

In [ ]:
c = 'Sold Price'
if c in final_df.select_dtypes('object').columns:
    final_df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)
final_df = final_df[(final_df['Sold Price'] >= 4 ) & (final_df['Sold Price'] <= 8 )]
final_df = final_df.dropna(subset=['Bedrooms'])

Evaluation metric

In [ ]:
def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

Prepare for test dataset and train dataset

In [ ]:
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2021, 1, 1)
final_df['Sold On'] = pd.to_datetime(final_df['Sold On'], errors='coerce')
train = final_df[(final_df['Sold On'] >= train_start) & (final_df['Sold On'] < test_start)]
test = final_df[(final_df['Sold On'] >= test_start) & (final_df['Sold On'] < test_end)]
train.shape, test.shape

Train

In [ ]:
from autogluon.tabular import TabularPredictor
    
label = 'Sold Price'    
predictor = TabularPredictor(label=label).fit(train)

Evaluate

In [ ]:
predictor.leaderboard(test, silent=True)

In [ ]:
predictor.feature_importance(test)

In [ ]:
preds = predictor.predict(test.drop(columns=[label]))
rmsle(preds, test[label])